## ЗАДАНИЕ ДЛЯ ПРОГРАММИРУЮЩИХ

**Ссылка**, на источник текста

In [296]:
#DATA_URL = "http://az.lib.ru/t/tolstoj_a_k/text_0180.shtml" # Train Data. Comments are made for this dataset
DATA_URL = "http://az.lib.ru/g/gogolx_n_w/text_0050.shtml" # My test data

Устанавливаем библиотеки

In [297]:
import warnings
warnings.filterwarnings('ignore')
!pip -q install rnnmorph

Создаём объект морфологического анализатора `RNNMorph`

In [298]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

Скачиваем текст, по которому будет дано задание, с помощью `urllib`

In [299]:
import urllib.request

opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset()) #Текс с html тегами

In [300]:
raw_text[:200]

'<html>\r\n<head>\r\n<title>Lib.ru/Классика: Гоголь Николай Васильевич. Вий</title>\r\n</head>\r\n\r\n<body>\r\n\r\n\r\n<center>\r\n\r\n<h2><a href=/g/gogolx_n_w/>Гоголь Николай Васильевич</a><br>\r\nВий</h2>\r\n\r\n<!------- П'

Как видно, текст содержит html теги, от которых нужно избавиться. Выбрасываем из текста HTML-теги с помощью библиотеки Beatiful soap

In [301]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(raw_text, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
cleaned_text = soup.get_text()

In [302]:
cleaned_text[:200]

'\n\nLib.ru/Классика: Гоголь Николай Васильевич. Вий\n\n\n\nГоголь Николай Васильевич\r\nВий\n\n\nLib.ru/Классика:\n\r\n\n\n[Регистрация]\n \n\r\n\r\n\r\n[Найти] \r\n[Рейтинги]\r\n[Обсуждения]\r\n[Новинки]\r\n[Обзоры]\r\n[Помощь]\r\n\r\n\n\n'

С помощью библиотеки [NLTK](https://nltk.org/) разбиваем текст на предложения и токены.

In [303]:
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')

tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
"A total of %d 'sentences'" % len(tokenized_sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


"A total of 934 'sentences'"

## Задание 1
С помощью метода `str.isalpha` из стандартной библиотеки Python модифицируйте нижеследующий код так, чтобы в predictions остались только буквенные токены.

In [304]:
from tqdm import tqdm
predictions = [[pred.normal_form for pred in sent]
               for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences") ]
for i, el in enumerate(predictions):
  predictions[i] = (list(filter(lambda sentencess : sentencess.isalpha(), el)))

15/15 [==============================] - 9s 488ms/step


sentences: 100%|██████████| 934/934 [00:00<00:00, 129332.45it/s]


Проверьте себя. Должны получиться следующие значения:

*   Предложений: 577 (возможны расхождения в несколько предложений)
*   Токенов: примерно 8621 (возможны расхождения в некоторое количество токенов)

In [305]:
len(predictions)

934

In [306]:
non_uniq_tokens = [word for sentence in predictions for word in sentence]
len(non_uniq_tokens)


11732

Для продолжения работы над заданием числа должны быть близки к указанным

## Задание 2

Используя `non_uniq_tokens`, стоп-слова для русского языка из библиотеки nltk (`nltk.corpus.stopwords`) и `nltk.FreqDist`, вычислите, **какую долю среди 100 самых частотных** токенов в произведении занимают токены, **не относящиеся** к стоп словам.

**Например**, если среди 100 самых частотных слов встречается 25 слов, входящих в стоп лист, значит не входят в стоп лист 75 слов, и их доля составит 0.75.

**Не бойтесь использовать документацию NLTK и тьюториалы.**

In [312]:
import nltk
from nltk import FreqDist
from nltk.corpus import stopwords
from functools import reduce
nltk.download("stopwords")
STOPWORDS = set(stopwords.words("russian"))
print(stopwords.words("russian")[:5]) #Пример стоп слов

n_amount_of_most_frequent = 50

predictions_flatten = reduce(lambda x, y : x+y, predictions)
freq_dist = FreqDist(w.lower() for w in predictions_flatten).items()
all_words = FreqDist(w.lower() for w in predictions_flatten).most_common(n_amount_of_most_frequent)
bad_words = FreqDist(w.lower() for w in predictions_flatten if w in STOPWORDS).items()

all_words_set = []
all_words_amount = 0
bad_words_set = []
bad_words_amount = 0

for k, v in all_words:
  all_words_set.append(k)
  all_words_amount+=v
for k, v in bad_words:
  bad_words_set.append(k)
  bad_words_amount+=v

all_words_set = set(all_words_set)
bad_words_set = set(bad_words_set)

print(len(all_words_set - bad_words_set)/ len(all_words_set))






['и', 'в', 'во', 'не', 'что']
0.28


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Проверьте себя: должно получиться 0.49 (допустимо небольшое расхождение)

## Задание 3
Вычислите, сколько токенов встречается в тексте **строго больше** 50 раз.

In [313]:
amount_of_tokens = 0
for k, v in freq_dist:
  if v > 50:
    amount_of_tokens+=1
amount_of_tokens

29

Проверьте себя: должно получиться значение 22 (возможно небольшое расхождение)
